## Chapter 10

### Exercise 1

In [1]:
import time
from dsets import LunaDataset

def time_iterations(iter_count):
    ds = LunaDataset()

    start = time.time()
    for i in range(iter_count):
        var = ds[i]
    end = time.time()

    print(f'{iter_count} iterations finished in {end - start} seconds.')

def time_last_iterations(iter_count):
    ds = LunaDataset()

    start = time.time()
    for i in range(iter_count):
        var = ds[-i]
    end = time.time()

    print(f'{iter_count} iterations finished in {end - start} seconds.')

In [4]:
time_iterations(1000)

1000 iterations finished in 186.75063300132751 seconds.


a) First run finished in 141 seconds.

In [5]:
time_iterations(1000)

1000 iterations finished in 0.7033522129058838 seconds.


b) Second run finished in under 1 second.

In [6]:
time_iterations(1000)

1000 iterations finished in 208.80103278160095 seconds.


c) After clearing the cache, the runtime is back to 200 seconds.

In [8]:
time_last_iterations(1000)
time_last_iterations(1000)

1000 iterations finished in 2.2063560485839844 seconds.
1000 iterations finished in 0.22845101356506348 seconds.


d) Using the last 1000 samples has no impact on the runtime after being cached.

### Exercise 2

In [2]:
time_iterations(1000)
time_iterations(1000)

1000 iterations finished in 225.06759643554688 seconds.
1000 iterations finished in 211.10741472244263 seconds.


After randomizing the list, both runs take quite a long time.

### Exercise 3

In [2]:
time_iterations(1000)
time_iterations(1000)

New instance of LunaDataset created.
1000 iterations finished in 278.02917885780334 seconds.
New instance of LunaDataset created.
1000 iterations finished in 0.6841669082641602 seconds.


The getCt decorator does have an impact on the first loop. The second one remains the same, however.